# TF-IDFのサンプル

TF-IDF(Term Frequency and Inverse Document Frequency)とは、単語の「その文章内での出現頻度」と「文章全体での出現頻度」に着目した方法です。  
「その文章内での出現頻度」が高く、かつ「文章全体での出現頻度」が低い単語についてスコアが高くなる仕組みになっています。  
このことから、高いスコアの単語は「その文章の特徴となる」ものであると考えられます。  
また、特定の単語のスコアが高い傾向にある文章をリストアップすると、それらは「似た内容の文章」であると考えられます。

## ファイルを読み込み単語リストを生成

事前準備としてMeCabをインストールしておきます。  
※MeCabは文章を単語に分解するために使用します。

In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

ローカルから分析対象となるテキストファイルをアップロードします。  
アップロードしたテキストファイルの内容を読み出して、単語に分解します。

In [2]:
import MeCab
from gensim.models.doc2vec import TaggedDocument

from google.colab import files

# ファイル名と、そのファイルに含まれる単語一覧を保持する配列
file_words_lists = []

# ローカルから文章ファイルをアップロード(複数選択可能)
uploaded = files.upload()

# MeCabインスタンスを生成
# 単語分解した際の出力フォーマットを"chasen"形式にする
mecab = MeCab.Tagger("-Ochasen")

for file_name in uploaded.keys():
  # 文章から抽出した単語の一覧
  words = []

  # ファイルから読み出した文章をfile_contentに格納
  # TODO: エンコード形式は実際の使用ケースに合わせて変更する
  file_content = uploaded[file_name].decode('sjis')

  # 文章を単語に分解(単語ごとに1行の分解結果が出力される)
  lines = mecab.parse(file_content).splitlines()
  for line in lines:
    chunks = line.split('\t') # 分解結果の項目はタブで区切られている
    # TODO: 抽出する品詞は要調整
    # 分解結果を確認し、動詞・形容詞・名詞(数を除く)のみ抽出
    if len(chunks) > 3 \
        and (chunks[3].startswith('動詞') or chunks[3].startswith('形容詞') \
             or (chunks[3].startswith('名詞') and not chunks[3].startswith('名詞-数'))):
      words.append(chunks[0])
  
  # ファイル名と、そこから抽出した単語一覧をセットにして登録
  file_words_lists.append(TaggedDocument(words=words, tags=file_name))
  print(file_name, words)

Saving l01.txt to l01.txt
Saving l02.txt to l02.txt
Saving l03.txt to l03.txt
Saving l04.txt to l04.txt
Saving l05.txt to l05.txt
Saving l06.txt to l06.txt
Saving l07.txt to l07.txt
Saving l08.txt to l08.txt
Saving l09.txt to l09.txt
Saving l10.txt to l10.txt
Saving l11.txt to l11.txt
Saving l12.txt to l12.txt
Saving l13.txt to l13.txt
Saving l14.txt to l14.txt
Saving l15.txt to l15.txt
Saving l16.txt to l16.txt
Saving l17.txt to l17.txt
Saving l18.txt to l18.txt
Saving l19.txt to l19.txt
Saving l20.txt to l20.txt
Saving l21.txt to l21.txt
Saving l22.txt to l22.txt
Saving l23.txt to l23.txt
Saving l24.txt to l24.txt
Saving l25.txt to l25.txt
Saving l26.txt to l26.txt
Saving l27.txt to l27.txt
Saving l28.txt to l28.txt
Saving l29.txt to l29.txt
Saving l30.txt to l30.txt
Saving l31.txt to l31.txt
Saving l32.txt to l32.txt
Saving l33.txt to l33.txt
Saving l34.txt to l34.txt
Saving l35.txt to l35.txt
Saving l36.txt to l36.txt
Saving l37.txt to l37.txt
Saving l38.txt to l38.txt
Saving l39.t

今回は第一生命の「よくあるご質問」から「保険金・給付金・年金などのご請求 」の質問文を使用しました。  
http://www.qa.dai-ichi-life.co.jp/category/show/4  
単語の分解精度は使用する辞書に依存します。  
上記だと「保険金」が分かれているのがわかります。  
この辺りは辞書にカスタマイズを加えて実用に堪えるものにしてい必要がありそうです。

## 単語と単語IDを相互に参照するための辞書を作成

単語に分解したらTF-IDFの算出をしていきます。  
TF-IDFを含めて、自然言語解析にはgensimというライブラリを使用すると便利です。  
まずは機械が処理をしやすいように、単語にIDを振り辞書を作成します。  
(下記のコードではdictionary)  
この後の処理では単語はすべて単語IDで示されるため、IDから単語を引くための逆引き辞書もあわせて作成しておきます。  
(下記のコードではdictionary_reverse)

In [0]:
from gensim import corpora

# 抽出したすべての単語を集約したリストを作成
all_words = []
for file_words in file_words_lists:
  all_words.append(file_words.words)

# 単語にID化し重複を除外した辞書を作成
dictionary = corpora.Dictionary(all_words)

# IDに対する単語を登録する辞書を作成
dictionary_reverse = {}
for dic in dictionary.token2id.items():
    dictionary_reverse[dic[1]]=dic[0]

## 文章に含まれる各単語のTF-IDFを算出

まずは単語の出現回数をカウントしていきます。  
その後、TF-IDFの算出のため、特定の文章内および文章全体での単語の出現回数をカウントします。  
TF-IDF算出後、そのままの状態(corpus_tfidf)だと「単語ID, TF-IDF値」の組み合わせになっているため、先に生成した逆引き辞書を使って「単語, TF-IDF値」に変換(texts_tfidf)します。  
分析結果として、ファイル名と、TF-IDF値の大きい単語を出力します。


In [4]:
from gensim import models
from operator import itemgetter

# 文章ごと含まれる単語idの個数を算出
corpus = list(map(dictionary.doc2bow,all_words))

# tfidf modelの生成
test_model = models.TfidfModel(corpus)

# 文章に含まれる単語のtf-idfを算出
corpus_tfidf = test_model[corpus]
# id->単語へ変換
texts_tfidf = [] # id -> 単語表示に変えた文章ごとのTF-IDF
for doc in corpus_tfidf:
    text_tfidf = []
    for word in doc:
        text_tfidf.append([dictionary_reverse[word[0]],word[1]])
    texts_tfidf.append(text_tfidf)

# 表示
print('===結果表示===')
for (file_words, vector) in zip(file_words_lists, texts_tfidf):
    print(file_words.tags) # ファイル名を出力
    # TF-IDF値の大きい順に単語を並べ替え
    sorted_vector = sorted(vector, key=itemgetter(1), reverse=True)
    for i in range(2): # TODO: 出力する数については文章に含まれる単語数で調整
        print(sorted_vector[i])

===結果表示===
l01.txt
['れる', 0.6262731805335507]
['支払わ', 0.5200684635543134]
l02.txt
['手術', 0.6354835273038828]
['日帰り', 0.5345757730184485]
l03.txt
['方法', 0.7567426791923162]
['ください', 0.3985333766649167]
l04.txt
['とき', 0.642987712597342]
['れる', 0.4711521793599111]
l05.txt
['取り付ける', 0.3816376082404674]
['病院', 0.3816376082404674]
l06.txt
['誰', 0.7487037623923931]
['すれ', 0.39429973589759393]
l07.txt
['かかる', 0.5430966008224561]
['税金', 0.5430966008224561]
l08.txt
['取扱', 0.3638238902636802]
['地域', 0.3638238902636802]
l09.txt
['振り込ま', 0.5844003882682005]
['い', 0.5150695176214285]
l10.txt
['満期', 0.7429800893693026]
['受け取り', 0.5903754809639442]
l11.txt
['入院', 0.7075152049205389]
['中', 0.5330126073193536]
l12.txt
['振り込ま', 0.6378359556499866]
['い', 0.5621657079520426]
l13.txt
['代理', 0.5832870196255684]
['特約', 0.4274062242633123]
l14.txt
['ない', 0.6157387156368268]
['証券', 0.6157387156368268]
l15.txt
['金額', 0.6259161334034955]
['満期', 0.49897444283012027]
l16.txt
['状況', 0.6827448961430665]
['確認', 0.5002

今回は、文章に含まれる単語数の最小が2(l24.txt)だったため、TF-IDF値が大きい順に2つの単語を出力しています。  
似ている文章を探す際には、特定の文章のTF-IDF値の大きい単語について、他の文章でもTF-IDF値が大きくなっていれば、似ていると考えることができます。  
上記の結果でいうと、例としては「受け取り」を含む以下の3つの文章が似ていることは、感覚的に納得できると思います。  
l10.txt 「満期保険金を受け取りたい。」  
l18.txt 「生存保険金を受け取りたい。」  
l24.txt 「年金を受け取りたい。」